In [1]:
import os
from google import genai
from google.genai import types
import pathlib
import wave
import chunked_tts

In [2]:
FILEPATH = pathlib.Path('/Users/Itai/Downloads/s41586-025-09442-9_reference.pdf')
PPMT = 10 # price per million tokens for audio generation in dollars. Used to set a price cap for the audio generation.
PRICE_CAP = 0.5 # price cap for the audio generation in dollars.
OUTPUT_FILE = 'saved_paper.wav'
gemini_api_key = os.environ['GEMINI_API_KEY']
client = genai.Client()

In [ ]:
# Retrieve and encode the PDF byte
prompt = "Print out all of the text in the paper that a narrator would read aloud. Include the title and section headings."+\
         "Do not include citation numbers. Do not include methods, acknowledgements, bibliography,"+\
         "reporting summary, or competing interests."

parsed_response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=FILEPATH.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])


In [ ]:
text_token_count = client.models.count_tokens(
    model="gemini-2.5-flash-preview-tts", contents=parsed_response.text
)

cost_estimate = PPMT*text_token_count.total_tokens/1000000
print(text_token_count)
print(f'estimated cost = {cost_estimate}$')

In [ ]:
if cost_estimate < PRICE_CAP:
   chunks = chunked_tts.chunk_text_by_lines(parsed_response.text)
   print(f"Splitting input into {len(chunks)} chunks…")

   pcm_all = b""
   for idx, chunk in enumerate(chunks, 1):
         print(f"→ Processing chunk {idx}/{len(chunks)} (chars={len(chunk)})")
         pcm_all += chunked_tts.tts_chunk(client, chunk)

   chunked_tts.wave_file(OUTPUT_FILE, pcm_all)
   print(f"Wrote {OUTPUT_FILE} (size={len(pcm_all)/24000/2:.1f} s)")

In [ ]:
parsed_response.text[9408+9400:9466+9409]